# TP3 & 4 - Artifical Intelligence

#### Marlon KUQI & Théo MARIE

### 1 - Recurrent neural network / LSTM : IMDB sentiment classification

![Encoders/Decoders for LTSM](https://qph.fs.quoracdn.net/main-qimg-febee5b881545802a75c064a84ecf85d)

In [2]:
'''
#Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
**Notes**
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

# ---------- Data preparation -----------

# Loads data from the imdb dataset
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

# Reshapes the training and testing sets
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

# ---------- Model building -----------

print('Build model...')
model = Sequential()

# The Embedding layer indicates that we work on a vocabulary (i.e. distinct words) of size "max_features"
# Our embedding vectors will have a size of 128
model.add(Embedding(max_features, 128))

# The LSTM layer will have an output size of 128
# "dropout" -> Fraction of the units to drop for the linear transformation of the inputs.
# "recurrent_dropout" -> Fraction of the units to drop for the linear transformation of the recurrent state.
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

# The last layer of our NN will be a regular one, of size 1 and activated by a sigmoid function
model.add(Dense(1, activation='sigmoid'))

# Compiles the model
# "loss" -> Loss function used, here binary_crossentropy (i.e. LogLoss)
# "optimizer" -> Optimizer for the convergence of the loss function, here adam. We used SGD in the last TPs
# "metrics" -> List of metrics to be evaluated during testing and training
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Using TensorFlow backend.


Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 80s 3ms/step - loss: 0.4633 - acc: 0.7804 - val_loss: 0.3843 - val_acc: 0.8317
Epoch 2/15
25000/25000 [==============================] - 82s 3ms/step - loss: 0.3065 - acc: 0.8734 - val_loss: 0.3762 - val_acc: 0.8314
Epoch 3/15
25000/25000 [==============================] - 83s 3ms/step - loss: 0.2212 - acc: 0.9126 - val_loss: 0.4469 - val_acc: 0.8295
Epoch 4/15
25000/25000 [==============================] - 81s 3ms/step - loss: 0.1552 - acc: 0.9418 - val_loss: 0.4580 - val_acc: 0.8322
Epoch 5/15
25000/25000 [==============================] - 81s 3ms/step - loss: 0.1054 - acc: 0.9619 - val_loss: 0.5490 - val_acc: 0.8184
Epoch 6/15
25000/25000 [==============================] - 77s 3ms/step - loss: 0.0841 - acc: 0.969

#### Our first results give the following figures:

Test score: 1.1003549643659591

Test accuracy: 0.8094

### 2 - Text classification: the Ohsumed dataset

In [38]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict

#### Number of files for each category

In [92]:
categories = 23

training_path = "ohsumed-first-20000\\training\C"
test_path = "ohsumed-first-20000\\test\C"

filecount = defaultdict(int)

for i in range(1,categories+1) :
    if (i < 10) :
        current_category = str("0" + str(i))
    else :
        current_category = str(i)
        
    for root, dirs, files in os.walk(training_path + current_category):  
        filecount[current_category] += len(files)
            
    for root, dirs, files in os.walk(test_path + current_category):  
        filecount[current_category] += len(files)

print("CAT  NB OF FILES")
print("----------------")
for key, value in filecount.items():
    print("C" + str(key) + "\t" + str(value))

CAT  NB OF FILES
----------------
C01	929
C02	391
C03	135
C04	2630
C05	712
C06	1220
C07	246
C08	1073
C09	254
C10	1562
C11	364
C12	1039
C13	667
C14	2550
C15	535
C16	428
C17	643
C18	788
C19	382
C20	1220
C21	1263
C22	183
C23	3952


#### Frequency of words in the C03 category

In [54]:
current_category = "03"

training_path = "ohsumed-first-20000\\training\C"
test_path = "ohsumed-first-20000\\test\C" 

wordcount = defaultdict(int)

for root, dirs, files in os.walk(training_path + current_category):  
    for filename in files:
        # print(filename)
        with open(training_path + current_category + "\\" + filename) as file:
            for word in file.read().split():
                wordcount[word] += 1

    for k,v in sorted(wordcount.items(), reverse=True, key=lambda x: x[1]):
        print (k, v)

of 516
the 464
in 309
and 296
with 188
to 182
a 133
were 116
was 110
for 90
from 72
patients 67
by 65
is 58
The 54
or 52
that 41
as 36
at 35
infection 35
than 34
be 31
after 30
on 30
are 30
disease 27
not 27
In 27
an 26
A 26
this 25
cases 24
had 23
falciparum 22
treated 21
levels 21
treatment 20
cerebral 20
clinical 20
all 19
We 19
patient 18
these 18
malaria 18
no 17
2 17
Plasmodium 17
drug 16
isolates 16
1 15
diagnosis 15
but 15
parasite 15
antibody 15
only 15
study 15
studies 14
been 14
most 14
during 14
blood 14
rosetting 14
less 14
day 13
children 13
may 13
parasites 13
antibodies 13
leishmaniasis 13
days 13
other 13
3 12
between 12
disease. 12
transmission 12
antigens 12
against 12
acquired 12
infections 12
infected 12
associated 12
significantly 12
months 11
compared 11
there 11
lesions 11
found 11
present 11
pulmonary 11
Toxoplasma 11
control 11
peptides 11
response 11
sera 11
case 11
which 11
greater 11
C. 11
5 11
renal 11
positive 10
before 10
However, 10
have 10
infection. 1

monitored 2
mated 2
pups, 2
1,000 2
regimen; 2
matings 2
manifested 2
edema. 2
enlargement 2
sibling 2
cardiopathy. 2
physical 2
residence 2
developing 2
seropositive 2
normal 2
subgroups 2
abnormalities. 2
HHD 2
anterior 2
seronegative 2
sodium 2
recommended 2
antimony, 2
days. 2
12-month 2
moderate 2
nasal 2
mucosa). 2
examinations, 2
regimen 2
patients), 2
abnormal 2
(10 2
pain 2
suggests 2
Acute 2
130 2
surgery 2
hypotension 2
relief 2
jaundice. 2
crystalluria 2
encephalitis 2
syndrome, 2
urinary 2
toxic 2
must 2
aware 2
injury 2
anemia. 2
Numerous 2
anemia 2
lindane, 2
literature. 2
evidence 2
chagasic 2
pathophysiological 2
result 2
cardiomyopathic 2
progression 2
onchocercal 2
57 2
detailed 2
fundus 2
pigment 2
edge 2
chorioretinal 2
progress 2
retina 2
retinochoroiditis 2
scan 2
nerve. 2
eye 2
brucellosis. 2
alert 2
central 2
nervous 2
parts 2
scoleces 2
report 2
rare 2
echinococcosis 2
Indians 2
Caribbean 2
standard 2
neurologic 2
seizures 2
primarily 2
electron 2
microscopy. 

S. 1
aureus, 1
investigation 1
sought 1
Entamoeba 1
223 1
school 1
Four 1
coproparasitoscopic 1
examination: 1
(20%); 1
parasite-free 1
(25%). 1
accuracy 1
extract 1
91.5% 1
(sensitivity, 1
85%; 1
specificity, 1
98%), 1
.753, 1
.001) 1
fecal 1
activity. 1
Measurement 1
diagnosing 1
wide 1
range 1
Moreover, 1
non 1
immunoepidemiologic 1
surveys. 1
Effects 1
jird. 1
Albendazole 1
medicated 1
durations. 1
mortality, 1
metastases 1
final 1
appearance 1
tissue. 1
Viability 1
Albendazole-medicated 1
(0.05%-0.10%) 1
inhibited 1
degeneration 1
necrosis, 1
protoscolex 1
formation, 1
metastases, 1
mortality 1
hosts. 1
Adverse 1
metabolite 1
exceeding 1
achieved 1
kill 1
Dexamethasone 1
increases 1
albendazole. 1
Therapy 1
complicated 1
exacerbation 1
follows 1
destruction 1
cysticerci. 1
adverse 1
effective. 1
shown 1
lowers 1
praziquantel, 1
thus 1
efficacy. 1
albendazole, 1
strong 1
drug, 1
simultaneously. 1
0.002); 1
hence, 1
ensuing 1
simultaneously 1
diminishing 1
drug. 1
Toxoplasmosis 1
Ca

#### Frequency of words in all categories

In [ ]:
categories = 23

training_path = "ohsumed-first-20000\\training\C"

wordcount = defaultdict(int)

for i in range(1,categories+1) :
    if (i < 10) :
        current_category = "0"+str(i)
    else :
        current_category = str(i)
        
    for root, dirs, files in os.walk(training_path + current_category):  
        for filename in files:
            # print(filename)
            with open(training_path + current_category + "\\" + filename) as file:
                for word in file.read().split():
                    wordcount[word] += 1
        
sorted_dict = sorted(wordcount.items(), reverse=True, key=lambda x: x[1])
        
for k,v in sorted_dict:
    print (k, v)

of 83629
the 71204
and 56493
in 50392
with 30321
to 29578
a 26048
was 19195
patients 17871
were 16826
for 14563
The 13663
is 10583
that 10105
by 9959
or 9951
than 8596
from 7478
be 7065
an 6509
as 6445
had 6427
on 6150
at 5998
after 5499
not 5378
are 5047
less 4642
In 4498
this 4355
+/- 4123
have 3812
A 3791
may 3731
= 3553
We 3374
but 3332
these 3166
treatment 3124
during 3123
who 3059
disease 3028
between 2903
been 2881
no 2804
clinical 2740
more 2677
group 2673
two 2662
patient 2611
associated 2588
blood 2574
which 2505
study 2498
This 2475
has 2407
one 2361
normal 2309
all 2303
significantly 2289
cell 2234
both 2233
results 2158
increased 2149
cases 2078
other 2071
significant 2064
compared 2057
cells 1958
also 1941
human 1934
found 1931
risk 1930
only 1910
control 1873
those 1859
acute 1827
These 1819
patients. 1791
three 1766
rate 1757
levels 1735
2 1725
can 1710
1 1698
treated 1693
(p 1683
used 1660
showed 1649
therapy 1649
their 1640
greater 1623
coronary 1599
without 1586
year

continued 197
clearance 197
pregnant 197
histamine 197
developing 197
success 197
obstructive 197
residual 197
culture 196
modified 196
Six 196
transient 196
intraoperative 196
differentiation 196
asthma 196
responsible 195
rheumatoid 195
mg/kg 195
dissection 195
every 194
maximum 194
enhanced 194
(greater 194
rupture 194
flap 194
colon 194
lymphocytes 193
chromosome 193
diffuse 192
little 192
amount 192
stable 191
United 191
assay 191
transmission 191
circulating 191
fatty 191
prospectively 190
recognized 190
altered 190
child 190
origin 190
molecular 190
temporal 190
intracranial 189
external 189
0.01) 189
alternative 189
safe 189
uptake 189
situ 189
genes 189
recorded 189
ethanol 189
fat 189
sinus 188
highest 188
vessels 188
almost 188
colorectal 188
detect 187
reaction 187
activity. 187
predictive 187
diagnosis. 186
By 186
dialysis 186
glomerular 186
dependent 185
them 185
reactions 185
capacity 185
individual 185
maintained 185
2) 184
Serum 184
condition 184
0.001) 184
creatinine 

tubular 97
polyps 97
he 97
whole 97
high-risk 97
arthroplasty 97
B, 97
special 97
assessing 97
preliminary 97
permanent 97
70% 97
47 97
analysis, 97
67 97
response. 97
antagonists 97
imaging. 97
contralateral 97
NIDDM 97
regurgitation 96
episode 96
explain 96
account 96
spread 96
cure 96
close 96
cartilage 96
observation 96
controls, 96
contain 96
diarrhea 96
1989, 96
metastasis 96
reflect 96
unchanged 96
otherwise 96
Moreover, 96
rats, 96
irradiation 96
phenotype 96
blockers 96
tumours 96
tumor, 96
islet 96
infarction, 96
morphine 96
knowledge, 95
400 95
yielded 95
immunologic 95
profile 95
examinations 95
restriction 95
ear 95
recommend 95
revascularization 95
thought 95
procedures. 95
locus 95
burn 95
medial 95
fibroblasts 95
stone 95
amyloid 95
resting 95
easily 94
weight, 94
definite 94
lactate 94
arthritis. 94
lymphoma. 94
abuse 94
unrelated 94
carinii 94
polymerase 94
favorable 94
computerized 94
reduces 94
outcomes 94
preservation 94
norepinephrine 94
biologic 94
morphologic 94

morphology 64
restored 64
immunization 64
measles 64
seronegative 64
gel 64
Southern 64
dexamethasone 64
ratio, 64
geriatric 64
Sixteen 64
follow 64
probable 64
subjective 64
gamma 64
GH 64
displacement 64
somatostatin 64
(6 64
prostatectomy 64
IGF-I 64
acetylcholine 64
propranolol 64
bacteremia 63
milk 63
discharged 63
trachomatis 63
dosage 63
treatments 63
immunocompromised 63
He 63
confined 63
4, 63
1% 63
AIDS, 63
regardless 63
test. 63
near 63
mode 63
fifth 63
creatine 63
behavioral 63
cognitive 63
wounds 63
F1 63
thermal 63
arm 63
Additional 63
starting 63
regulatory 63
diagnoses 63
injections 63
adenomatous 63
hand, 63
malformations 63
saphenous 63
finger 63
RV 63
parenteral 62
readily 62
focus 62
radiographs 62
criteria. 62
fecal 62
obtain 62
ultrastructural 62
pigs 62
proper 62
limbs 62
immunofluorescence 62
correlates 62
atopic 62
2), 62
response, 62
Those 62
disease: 62
soft-tissue 62
AIDS-related 62
detailed 62
complaints 62
2.0 62
Computed 62
Risk 62
estimate 62
(from 62
sp

Malignant 43
Twenty-six 43
pathologically 43
took 43
140 43
middle-aged 43
cm. 43
dissection. 43
laparotomy 43
hysterectomy 43
created 43
connection 43
PLP 43
missed 43
normal, 43
age- 43
necrotic 43
infertile 43
tracheostomy 43
avascular 43
velocities 43
shunting 43
enteric 42
pylori. 42
barrier 42
pretreated 42
assay. 42
78 42
bipolar 42
systems. 42
She 42
E. 42
toxin 42
Peripheral 42
agreed 42
agent. 42
newer 42
tomography. 42
cirrhosis. 42
method, 42
principal 42
PAF 42
profiles 42
students 42
Quantitative 42
categories 42
97 42
take 42
apical 42
bronchitis 42
effusions 42
phospholipase 42
already 42
7, 42
trauma, 42
cases; 42
causes. 42
males, 42
increased, 42
valve. 42
isoelectric 42
focusing 42
(18 42
66 42
parasite 42
nitrogen 42
carry 42
intramuscular 42
mediator 42
hematocrit 42
utility 42
phosphorylation 42
gold 42
age-related 42
latency 42
adolescents 42
pattern, 42
layer 42
psoriasis. 42
HSV 42
gut 42
interpretation 42
ipsilateral 42
Measurement 42
bcl-2 42
precursor 42
di

...

foramina 6
(nasal 6
carcinoma), 6
1.0. 6
meta-analysis. 6
contraceptives, 6
44%. 6
74%. 6
SEC 6
TRUS 6
skin-infiltrating 6
(young 6
(midline 6
etiologically 6
Twin 6
statewide 6
Connecticut 6
encourages 6
Prompt 6
Understanding 6
29.4 6
carina 6
beta-OHA 6
prominent, 6
HLA-DR. 6
RNAs 6
average-risk 6
tibia. 6
ulcer: 6
transepithelial 6
Pathology 6
well-established 6
consultations 6
ECL-cell 6
maintained. 6
middle, 6
chain. 6
differences, 6
derivation 6
hypotonic 6
KCl, 6
eliminated, 6
ignored 6
glands; 6
triangle 6
lorazepam, 6
ml/min/m2) 6
antipyrine 6
244 6
fibromatous 6
(GM-CSF) 6
urea, 6
vacuolization 6
pemphigoid. 6
side-effects. 6
(89%), 6
behaves 6
junction, 6
Roux-en-Y 6
lithium, 6
myxedema 6
mounted 6
endoscope 6
anteriorly 6
Secretion 6
adrenalectomies 6
mol/L) 6
inspected 6
this. 6
trophic 6
N-ras 6
CAA 6
lithotomy 6
citrate, 6
.05, 6
included, 6
amplification. 6
demyelination 6
(D) 6
dermatoses. 6
pachyonychia 6
suppurativa. 6
moiety 6
retrocrural 6
was: 6
bilateral, 6
appl

200-239, 6
HHANES 6
Expert 6
testimony 6
malpractice 6
programs, 6
mediolytic 6
vented 6
14.0 6
NMR 6
Arrhythmias 6
reocclusion 6
ST/delta 6
obesity-related 6
periodicity 6
cracks 6
spasm. 6
culprit 6
pindolol 6
g/day 6
suddenly. 6
applies 6
full-dose 6
aorta; 6
quitting 6
fish-oil 6
Veneto 6
thiazides 6
anti-hypertensive 6
pace 6
hamsters 6
subdiaphragmatic 6
defibrillation. 6
replaced. 6
bioprosthesis 6
Hancock 6
Hemodynamically 6
TAR 6
dilatations. 6
Reactive 6
(cardiac 6
nicardipine. 6
Scatter 6
Penetrating 6
stents. 6
warfarin; 6
driver's 6
auscultatory 6
pulled 6
refilling 6
Anomalous 6
AL, 6
2086, 6
leftward 6
mumol, 6
CoQ10. 6
fueled 6
mmol/L). 6
PAODE. 6
alpha- 6
pmol 6
mg/dL. 6
xanthomata 6
MIBI 6
2.73 6
C-peptide, 6
chloride-stained 6
baboon 6
VB, 6
Reynolds 6
Assist 6
apheresis 6
Novacor 6
Biomedicus 6
pts), 6
pts) 6
smoke-like 6
Buerger's 6
WEB-2086 6
dipyridamole-induced 6
IHD/SUD 6
Pacific 6
2-hr 6
sarcoidal 6
GPI-linked 6
ACD 6
packs 6
ovalocytosis 6
panniculitis 6
lipo

heat-stable, 5
mCi 5
technetium-99 5
Intraperitoneal 5
Y-90 5
14.2% 5
leucovorin 5
twice, 5
procarbazine 5
foundation 5
Profound 5
encased 5
heat-labile 5
G2/M 5
(IgG 5
1.09 5
streptavidin 5
dosimetry 5
spatially 5
automatically 5
ate 5
(48 5
axis. 5
screening: 5
contradistinction 5
instrument. 5
Pedunculated 5
(MTX) 5
TMP-SMX 5
13-year 5
Insulinlike 5
IGF-I. 5
ER-positive 5
LED 5
(AZT) 5
Staurosporine 5
anti-idiotype 5
keyhole 5
limpet 5
Anomalies 5
break 5
repaired, 5
stent, 5
possible; 5
Neoadjuvant 5
(5.0 5
further. 5
(11%), 5
imipramine. 5
conduit. 5
closed. 5
expressed. 5
chi-square). 5
0.005, 5
fulguration 5
urgency 5
modality. 5
resumed 5
aldosteronism 5
428 5
586 5
588 5
1.11 5
IL2 5
analysing 5
went 5
cecum, 5
obstructions. 5
(above 5
(10.4%) 5
2.5-fold 5
Umbilical 5
(PTHrP), 5
1-34 5
125I-Tyr 5
degrees-PTHrP(1-34) 5
radioligand, 5
cartridges. 5
pmol/liter; 5
pmol/liter). 5
pmol/liter) 5
consulted 5
institution, 5
processes, 5
index) 5
tempered 5
3500 5
bronchoscope. 5
sedati

high-arched 5
fascicles 5
attentional 5
(Experiment 5
horizontally 5
playing 5
AI 5
tone, 5
corticotropin-releasing 5
E, 5
synchrony 5
(47%); 5
normotensive, 5
case-finding 5
sleep/wake 5
Carolina 5
psychosis 5
non-REM 5
skill 5
produces, 5
sex- 5
vestibulo-ocular 5
multifactorial, 5
hypertension: 5
nucleus, 5
Quantification 5
Difficulties 5
31.4 5
21.8 5
dissociation, 5
medetomidine 5
SAH. 5
ganglion. 5
metabolizers 5
fingers 5
131% 5
lightly 5
related, 5
metabolic, 5
trochanter 5
day-case 5
aortography. 5
rigidity, 5
dyskinesia, 5
filling, 5
cardiologist 5
bias, 5
deviation. 5
0.44% 5
trimester, 5
meningoencephalitis 5
(PAN)-like 5
leptomeninges, 5
PAN-like 5
oil. 5
intravitreal 5
conjunctiva 5
bent 5
randomization, 5
A-scan 5
Consistent 5
(32% 5
chamber, 5
hypotony 5
professionals, 5
palpebral 5
sexes. 5
303 5
.01 5
admissions, 5
perception. 5
vitreoretinopathy. 5
anecdotal 5
11p13 5
spretus 5
evolutionarily 5
initiation. 5
11.4% 5
microangiopathy. 5
territories. 5
osteosynthesis 5


infection? 4
dysmotility. 4
scope 4
Counseling 4
Immunopathology 4
virology 4
testable 4
theory. 4
technology. 4
(HTLV-I) 4
seroreactivity 4
SIVagm 4
(HIV-1). 4
featured 4
conferences. 4
(12) 4
Thirty-two, 4
0.0003). 4
Amount 4
(2.0% 4
7.5%) 4
5.0%) 4
27.5%) 4
20.0%) 4
chickenpox--cause 4
coincidence? 4
boy, 4
chickenpox. 4
coincidence, 4
Edinburgh 4
haemophilia 4
concentrations; 4
Annual 4
junior 4
Computer 4
disease-specific, 4
epitope, 4
Proliferation 4
mitogens, 4
immunopathogenic 4
Mama 4
Yemo 4
seropositives, 4
Years 4
21.3 4
PBMC, 4
431 4
(CAPD), 4
(4.3% 4
12/278) 4
(27-2,040). 4
(10/14) 4
(4/14) 4
statement 4
Immunocompromised 4
Interferon-alpha 4
hepatotoxicity. 4
Variable 4
smoking: 4
31.6 4
worms, 4
CT-densities 4
elsewhere 4
Onchocerca 4
eggs 4
Toxoplasmic 4
71-yr-old 4
transaminase, 4
transaminase. 4
1:4096, 4
Brucella, 4
Tularemia 4
seventy 4
(1.1%) 4
underestimate 4
jird 4
Parasite 4
burdens, 4
jirds. 4
grove 4
Chrysops 4
figure 4
larvae 4
also. 4
tachyzoites 4
loads, 4
